In [ ]:
# Correr solo al empezar
!pip install yfinance
!pip install pandas==0.25.1

In [ ]:
import yfinance as yf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None
pd.set_option('display.max_columns', 10)
pd.set_option('display.max_rows', 150)
pd.set_option('display.width', 1000)

resumenpicos = pd.DataFrame()
for i in (
"ALUA.BA", 
"BBAR.BA", 
"BMA.BA", 
"BYMA.BA", 
"CEPU.BA", 
"COME.BA", 
"CRES.BA", 
"CVH.BA", 
"EDN.BA", 
"HARG.BA", 
"PAMP.BA", 
"SUPV.BA", 
"TECO2.BA", 
"TGNO4.BA", 
"TGSU2.BA", 
"TRAN.BA", 
"TXAR.BA", 
"VALO.BA",
"YPFD.BA"):
  # Parámetros a definir al inicio
        # Sobre la bajada
  ticker = i            # Activo (En MERVAL, "ticker.BA", btc como "BTC-USD", el dolar es "USDARS=X" pero no corre bien porque no tiene volumen)
  startDate = "2019-04-23"      # Fecha de inicio
  endDate = "2020-04-23"        # Fecha de cierre

        # Sobre los picos
  lags = 20                     # ESTE ES EL PARÁMETRO MÁS IMPORTANTE. Cantidad de períodos hacia atrás y adelante que hacen falta para que sea considerado un quiebre
                                        # lags bajos: más riesgo de incluir picos que no son, la ventaja es no perdernos ninguno y tener tendencias nuevas.
                                        # lags altos: me aseguro que fueron picos reales, pero puede perderme alguno y todo tarda más en aparecer.
  volMin = 0.6                  # Percentil mínimo (en el período elegido) de volumen diario el día del pico, para que marque la tendencia

        # Sobre las tendencias
  minDurTendencia = 15         # Cantidad de períodos que podría extenderse una tendencia como minimo
  maxDurTendencia = 360         # Cantidad de períodos que podría extenderse una tendencia como máximo
                                        # Hacerlo más largo no debería afectar en nada al cálculo, sí podría a la performance

        # Sobre las rupturas
  intervaloPicos = 0.05            # Intervalo alrededor del pico proyectado en donde se considera que se rompió.
                                        # Tiene que ser positivo para que exista la Zona Precio y se pueda encontrar los rebotes.


  # Descarga de históricos

  df = yf.download(ticker, start=startDate, end=endDate)
  df['Date'] = df.index.values
  df['VolumePerc'] = pd.qcut(df.Volume, 100, labels=False, duplicates='drop')
  df['Rueda'] = np.arange(len(df))
  df = df[['Date', 'Rueda', 'Close', 'VolumePerc']]
  actualRueda = df.iloc[-1, df.columns.get_loc("Rueda")]
  actualPrice = round(df.iloc[-1, df.columns.get_loc("Close")],2)
  actualVolume = df.iloc[-1, df.columns.get_loc("VolumePerc")]


  # Construcción de variables y picos

  i = 1
  while i < (lags+1):
      colname = 'p%sb' % (i)                                                  
      df[colname] = round(df.Close.shift(i),2)
      j = i * -1
      colname = 'p%sf' % (-j)                                                  
      df[colname] = round(df.Close.shift(j),2)
      i = i + 1

  df['maxb'] = round(df.filter(regex=(".*b")).max(axis=1),2)
  df['maxf']= round(df.filter(regex=(".*f")).max(axis=1),2)
  df['minb'] = round(df.filter(regex=(".*b")).min(axis=1),2)
  df['minf'] = round(df.filter(regex=(".*f")).min(axis=1),2)
  df['techo'] = np.where((df['Close']>df['maxb']) & (df['Close']>df['maxf']), 1, 0)
  df['piso'] = np.where((df['Close']<df['minb']) & (df['Close']<df['minf']), 1, 0)
  i=minDurTendencia
  while i < maxDurTendencia:
      colname = 'pr%sf' % (i)                                                  
      df[colname] = round(df.Close.shift(-i),2)
      i = i + 1
  df['PrecioAdvertencia_Pico'] = np.int64(df.Close.shift(-lags))

  # Armo las proyecciones de los picos en caso de que la tendencia no se rompa

  techos = df[(df['techo']==1)]
  techos['PicoAnterior_Rueda'] = np.int64(techos.Rueda.shift(1))
  techos['PicoProyectado_Lags'] = techos.Rueda-techos.PicoAnterior_Rueda
  techos['PicoAnterior_Close'] = techos.Close.shift(1)
  techos['Pico_Pendiente'] = (techos.Close-techos.PicoAnterior_Close)/techos.PicoProyectado_Lags
  techos['Pico_Origen'] = techos.Close-techos.Rueda*techos.Pico_Pendiente
  techos = techos[['Date','Rueda', 'Close', 'VolumePerc', 'PicoProyectado_Lags', 'Pico_Origen', 'Pico_Pendiente']]
  techos = techos[techos.VolumePerc > 0]
  techos = techos.dropna()

  pisos = df[(df['piso']==1)]
  pisos['PicoAnterior_Rueda'] = np.int64(pisos.Rueda.shift(1))
  pisos['PicoProyectado_Lags'] = pisos.Rueda-pisos.PicoAnterior_Rueda
  pisos['PicoAnterior_Close'] = pisos.Close.shift(1)
  pisos['Pico_Pendiente'] = (pisos.Close-pisos.PicoAnterior_Close)/pisos.PicoProyectado_Lags
  pisos['Pico_Origen'] = pisos.Close-pisos.Rueda*pisos.Pico_Pendiente
  pisos = pisos[['Date','Rueda', 'Close', 'VolumePerc', 'PicoProyectado_Lags', 'Pico_Origen', 'Pico_Pendiente']]
  pisos = pisos[pisos.VolumePerc > 0]
  pisos = pisos.dropna()

  tendencias = pd.DataFrame()

  for index, row in techos.iterrows():
      rueda = int(row['Rueda'])
      nombretendencia = '%s-%sT' % (ticker, rueda)
      volume = row['VolumePerc']
      close = row['Close']
      date = row['Date']
      lagsOrigen = row['PicoProyectado_Lags']
      b = row['Pico_Origen']
      m = round(row['Pico_Pendiente'],2)
      n = 0
      precioproyectado = round(actualRueda * m + b,2)
      if df.Rueda.max() > rueda + lags:
          evolution = pd.DataFrame()
          evolution['ruedas'] = list(range(rueda + lags, df.Rueda.max()))
          evolution['pico'] = nombretendencia
          evolution['precioProyectado'] = row['Pico_Origen'] + row['Pico_Pendiente']*evolution.ruedas
          evolution = evolution.merge(df, how='left', left_on=['ruedas'], right_on=['Rueda'])
          evolution = evolution[['pico', 'Rueda', 'precioProyectado', 'VolumePerc','Close']]
          evolution.loc[(evolution['Close'] > evolution['precioProyectado'] * (1-intervaloPicos))&(evolution['Close'] < evolution['precioProyectado'] * (1+intervaloPicos)), 'ZonaPrecio'] = 1
          evolution.loc[(evolution['Close'] < evolution['precioProyectado'] * (1-intervaloPicos))|(evolution['Close'] > evolution['precioProyectado'] * (1+intervaloPicos)), 'ZonaPrecio'] = 0
          evolution.loc[(evolution['Close'] < evolution['precioProyectado'] * (1-intervaloPicos)), 'BajoZonaPrecio'] = 1
          evolution.loc[(evolution['Close'] > evolution['precioProyectado'] * (1-intervaloPicos)), 'BajoZonaPrecio'] = 0
          evolution.loc[(evolution['Close'] > evolution['precioProyectado'] * (1+intervaloPicos)), 'SobreZonaPrecio'] = 1
          evolution.loc[(evolution['Close'] < evolution['precioProyectado'] * (1+intervaloPicos)), 'SobreZonaPrecio'] = 0
          tendencias = tendencias.append(evolution) 
          if evolution['ZonaPrecio'].max() == 0 and evolution['SobreZonaPrecio'].max() == 0:
            new_row = {'Tendencia':nombretendencia, 'Pendiente':m, 'FechaConfirmacion':date, 'LagsIniciales':lagsOrigen, 'PrecioOrigen': close, 'VolumenOrigen': volume, 'PrecioAlerta': precioproyectado,'Pendiente':m, 'Status':'Techo no probado', 'Última_Rueda': actualRueda, 'Último_Precio': actualPrice, 'Último_Volumen': actualVolume}
            resumenpicos = resumenpicos.append(new_row, sort=True, ignore_index=True)
          elif evolution['SobreZonaPrecio'].max() == 0:
            evolution['diffPrevious'] = evolution['ZonaPrecio'].diff()
            veces = len(evolution[evolution['diffPrevious'] == 1])
            veces = np.where(veces==0, 1, veces)
            new_row = {'Tendencia':nombretendencia, 'Pendiente':m, 'FechaConfirmacion':date, 'LagsIniciales':lagsOrigen, 'PrecioOrigen': close, 'VolumenOrigen': volume, 'PrecioAlerta': precioproyectado, 'Pendiente':m, 'Status':'Techo probado %s veces y vigente' % (veces), 'Última_Rueda': actualRueda, 'Último_Precio': actualPrice, 'Último_Volumen': actualVolume}
            resumenpicos = resumenpicos.append(new_row, sort=True, ignore_index=True)
          else:
            firstSZP = (evolution['SobreZonaPrecio']==1).idxmax()
            desdePrimerSZP = evolution.loc[firstSZP:]
            resistencia = round(desdePrimerSZP['Close'].iloc[0]*(1-intervaloPicos),2)
            desdePrimerSZP.loc[(desdePrimerSZP['Close'] > resistencia)&(desdePrimerSZP['Close'] < (resistencia*(1+intervaloPicos))), 'ZonaResistencia'] = 1
            desdePrimerSZP.loc[(desdePrimerSZP['Close'] < resistencia)|(desdePrimerSZP['Close'] > (resistencia*(1+intervaloPicos))), 'ZonaResistencia'] = 0
            desdePrimerSZP.loc[(desdePrimerSZP['Close'] < resistencia), 'BajoResistencia'] = 1
            desdePrimerSZP.loc[(desdePrimerSZP['Close'] >= resistencia), 'BajoResistencia'] = 0
            if desdePrimerSZP['ZonaResistencia'].max() == 0 and desdePrimerSZP['BajoResistencia'].max() == 0:
              new_row = {'Tendencia':nombretendencia, 'Pendiente':0, 'FechaConfirmacion':date, 'LagsIniciales':lagsOrigen, 'PrecioOrigen': close, 'VolumenOrigen': volume, 'PrecioAlerta': resistencia, 'Status':'Techo roto, R%s no probada' % (resistencia), 'Última_Rueda': actualRueda, 'Último_Precio': actualPrice, 'Último_Volumen': actualVolume}
              resumenpicos = resumenpicos.append(new_row, sort=True, ignore_index=True)
            elif desdePrimerSZP['BajoResistencia'].max() == 0:
              new_row = {'Tendencia':nombretendencia, 'Pendiente':0, 'FechaConfirmacion':date, 'LagsIniciales':lagsOrigen, 'PrecioOrigen': close, 'VolumenOrigen': volume, 'PrecioAlerta': resistencia, 'Status':'Techo roto, R%s probada y vigente' % (resistencia), 'Última_Rueda': actualRueda, 'Último_Precio': actualPrice, 'Último_Volumen': actualVolume}
              resumenpicos = resumenpicos.append(new_row, sort=True, ignore_index=True)
            else:
              new_row = {'Tendencia':nombretendencia, 'Pendiente':0, 'FechaConfirmacion':date, 'LagsIniciales':lagsOrigen, 'PrecioOrigen': close, 'VolumenOrigen': volume, 'PrecioAlerta': precioproyectado, 'Status':'Tendencia muerta', 'Última_Rueda': actualRueda, 'Último_Precio': actualPrice, 'Último_Volumen': actualVolume}
              resumenpicos = resumenpicos.append(new_row, sort=True, ignore_index=True)

  for index, row in pisos.iterrows():
      rueda = int(row['Rueda'])
      nombretendencia = '%s-%sP' % (ticker, rueda)
      volume = row['VolumePerc']
      close = row['Close']
      date = row['Date']
      lagsOrigen = int(row['PicoProyectado_Lags'])
      b = row['Pico_Origen']
      m = round(row['Pico_Pendiente'],2)
      n = 0
      precioproyectado = round(actualRueda * m + b,2)
      if df.Rueda.max() > rueda + lags:
          evolution = pd.DataFrame()
          evolution['ruedas'] = list(range(rueda + lags, df.Rueda.max()))
          evolution['pico'] = nombretendencia
          evolution['precioProyectado'] = row['Pico_Origen'] + row['Pico_Pendiente']*evolution.ruedas
          evolution = evolution.merge(df, how='left', left_on=['ruedas'], right_on=['Rueda'])
          evolution = evolution[['pico', 'Rueda', 'precioProyectado', 'VolumePerc','Close']]
          evolution.loc[(evolution['Close'] > evolution['precioProyectado'] * (1-intervaloPicos))&(evolution['Close'] < evolution['precioProyectado'] * (1+intervaloPicos)), 'ZonaPrecio'] = 1
          evolution.loc[(evolution['Close'] < evolution['precioProyectado'] * (1-intervaloPicos))|(evolution['Close'] > evolution['precioProyectado'] * (1+intervaloPicos)), 'ZonaPrecio'] = 0
          evolution.loc[(evolution['Close'] < evolution['precioProyectado'] * (1-intervaloPicos)), 'BajoZonaPrecio'] = 1
          evolution.loc[(evolution['Close'] > evolution['precioProyectado'] * (1-intervaloPicos)), 'BajoZonaPrecio'] = 0
          evolution.loc[(evolution['Close'] > evolution['precioProyectado'] * (1+intervaloPicos)), 'SobreZonaPrecio'] = 1
          evolution.loc[(evolution['Close'] < evolution['precioProyectado'] * (1+intervaloPicos)), 'SobreZonaPrecio'] = 0
          if evolution['ZonaPrecio'].max() == 0 and evolution['BajoZonaPrecio'].max() == 0:
            new_row = {'Tendencia':nombretendencia, 'Pendiente':m, 'FechaConfirmacion':date, 'LagsIniciales':lagsOrigen, 'PrecioOrigen': close, 'VolumenOrigen': volume, 'PrecioAlerta': precioproyectado,'Pendiente':m, 'Status':'Piso no probado', 'Última_Rueda': actualRueda, 'Último_Precio': actualPrice, 'Último_Volumen': actualVolume}
            resumenpicos = resumenpicos.append(new_row, sort=True, ignore_index=True)
          elif evolution['BajoZonaPrecio'].max() == 0:
            evolution['diffPrevious'] = evolution['ZonaPrecio'].diff()
            veces = len(evolution[evolution['diffPrevious'] == 1])
            veces = np.where(veces==0, 1, veces)
            new_row = {'Tendencia':nombretendencia, 'Pendiente':m, 'FechaConfirmacion':date, 'LagsIniciales':lagsOrigen, 'PrecioOrigen': close, 'VolumenOrigen': volume, 'PrecioAlerta': precioproyectado, 'Pendiente':m, 'Status':'Piso probado %s veces y vigente' % (veces), 'Última_Rueda': actualRueda, 'Último_Precio': actualPrice, 'Último_Volumen': actualVolume}
            resumenpicos = resumenpicos.append(new_row, sort=True, ignore_index=True)
          else:
            firstBZP = (evolution['BajoZonaPrecio']==1).idxmax()
            desdePrimerBZP = evolution.loc[firstBZP:]
            soporte = round(desdePrimerBZP['Close'].iloc[0]*(1+intervaloPicos),2)
            desdePrimerBZP.loc[(desdePrimerBZP['Close'] < soporte)&(desdePrimerBZP['Close'] > (soporte*(1-intervaloPicos))), 'ZonaSoporte'] = 1
            desdePrimerBZP.loc[(desdePrimerBZP['Close'] > soporte)|(desdePrimerBZP['Close'] < (soporte*(1-intervaloPicos))), 'ZonaSoporte'] = 0
            desdePrimerBZP.loc[(desdePrimerBZP['Close'] > soporte), 'SobreSoporte'] = 1
            desdePrimerBZP.loc[(desdePrimerBZP['Close'] <= soporte), 'SobreSoporte'] = 0
            if desdePrimerBZP['ZonaSoporte'].max() == 0 and desdePrimerBZP['SobreSoporte'].max() == 0:
              new_row = {'Tendencia':nombretendencia, 'Pendiente':0, 'FechaConfirmacion':date, 'LagsIniciales':lagsOrigen, 'PrecioOrigen': close, 'VolumenOrigen': volume, 'PrecioAlerta': soporte,'Status':'Piso roto, S%s no probado' % (soporte), 'Última_Rueda': actualRueda, 'Último_Precio': actualPrice, 'Último_Volumen': actualVolume}
              resumenpicos = resumenpicos.append(new_row, sort=True, ignore_index=True)
            elif desdePrimerBZP['SobreSoporte'].max() == 0:
              new_row = {'Tendencia':nombretendencia, 'Pendiente':0, 'FechaConfirmacion':date, 'LagsIniciales':lagsOrigen, 'PrecioOrigen': close, 'VolumenOrigen': volume, 'PrecioAlerta': soporte,'Status':'Piso roto, S%s probado y vigente' % (soporte), 'Última_Rueda': actualRueda, 'Último_Precio': actualPrice, 'Último_Volumen': actualVolume}
              resumenpicos = resumenpicos.append(new_row, sort=True, ignore_index=True)
            else:
              new_row = {'Tendencia':nombretendencia, 'Pendiente':0, 'FechaConfirmacion':date, 'LagsIniciales':lagsOrigen, 'PrecioOrigen': close, 'VolumenOrigen': volume, 'PrecioAlerta': precioproyectado, 'Status':'Tendencia muerta', 'Última_Rueda': actualRueda, 'Último_Precio': actualPrice, 'Último_Volumen': actualVolume}
              resumenpicos = resumenpicos.append(new_row, sort=True, ignore_index=True)

resumenpicos['Diff'] = abs(resumenpicos.Último_Precio/resumenpicos.PrecioAlerta-1)
resumenpicos = resumenpicos[resumenpicos.PrecioAlerta > 0].sort_values(by=['Diff'])
resumenpicos = resumenpicos[['Tendencia','Pendiente', 'FechaConfirmacion', 'LagsIniciales', 'Status', 'PrecioAlerta', 'Último_Precio']][resumenpicos.Status != 'Tendencia muerta']

print(endDate)
print(resumenpicos)